### A few examples of writing functions recursively along with a dynamic programming approach

In [33]:
# Function to comput the nth fibonacci number
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# This works and has O(2^n) runtime (dictated by the number of calls made)
print(fibonacci(6))

8


In [24]:
# Dynamic programming involves recording a cache of simpler results that can be used to solve more complex calls.
def dp_fibonacci(n):
    cache = [0, 1]
    
    if n <= len(cache):
        return cache[n]
    
    for i in range(2, n+1):
        cache.append(cache[i-2] + cache[i-1])
    return cache[-1]

In [31]:
# This has a runtime of O(n) so can calculate much larger values than the recursive approach
print(dp_fibonacci(6))
print(dp_fibonacci(6000))  # Recursive function for n=600 crashes my computer. Dynamic method finds the 6,000th one in a fraction of a second

8
14909059039666631258104


### Oh no! A child is running up n stairs and can take steps of either 1, 2 or 3. Calculate the number of possible ways the child can run up the stairs.

In [52]:
# If n=1, then the number of ways is 1
# If n=2, then the number of ways is 2: (1+1 steps, 2 steps)
# If n=3, then the number of ways is 4(?): (1+1+1, 1+2, 2+1, 3) Will we take 1+2 = 2+1?

# If n=n+1 then the number of ways is the same as n steps + some extra amount corresponding to the last added step. This last
# step can be reached either with an additional step of size 1 from the nth configuration, additional steps of size (2, 1+1)
# from the n-2th configuration or additional steps of size (1+1+1, 1+2, 2+1, 3) from the n-3th configuation

# e.g. x[9] different ways of getting up the steps if n=9. If n=10, you have (x[9]+1) + (x[8]+2) + (x[7]+3)
# We know that x[0]=1, x[1]=1, x[2]=2, x[3]=4

def numberOfWays(n):
    cache = [1, 1, 2, 4]  # Initialise the base cases of there being 0, 1, 2 or 3 steps
    if n <= len(cache)-1:
        return cache[n]  # nth entry in the cache is the number of ways of getting up n stairs
    else:
        for n in range(4, n+1):
            #cache.append((cache[n-1]+1) + (cache[n-2]+2) + (cache[n-3]+4))
            cache.append(cache[n-1] + cache[n-2] + cache[n-3])
        return cache[-1]

print(numberOfWays(4))  # numberOfWays(10) = 274. Seems to work.

7
